In [8]:
import numpy as np
import pandas as pd

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [9]:
# 1. 아이리스 데이터 로드 + 판다스 DataFrame 변환
iris = load_iris()
X = iris.data                          # (150, 4) 특징(feature)
y = iris.target                        # (150,) 레이블(label: 0,1,2)

df = pd.DataFrame(X, columns=iris.feature_names) #컬럼이름에 피쳐네임 지정
df["target"] = y #예측변수 지정

print(df.head())






   sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)  \
0                5.1               3.5                1.4               0.2   
1                4.9               3.0                1.4               0.2   
2                4.7               3.2                1.3               0.2   
3                4.6               3.1                1.5               0.2   
4                5.0               3.6                1.4               0.2   

   target  
0       0  
1       0  
2       0  
3       0  
4       0  


In [11]:
# 2. 학습/테스트 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(
    df[iris.feature_names],
    df["target"],
    test_size=0.2,
    random_state=42,
    stratify=df["target"], #타겟을 기준으로 비율 지정
)


In [ ]:
# 3. 스케일링(Standardization)
scaler = StandardScaler() #scaler에 스케일링함수 저장
X_train_scaled = scaler.fit_transform(X_train) #X_train에 평균,편차 학습후 변환
X_test_scaled = scaler.transform(X_test) #X_test 테스트에 스케일링 변환



In [ ]:
# 4. 딥러닝 모델 정의 (입력 4차원 데이터, 3층 신경망 은닉층 2개, 출력1개 3클래스)
model = keras.Sequential([ #keras.Sequential 모델 지정
    layers.Input(shape=(4,)),          # 꽃받침/꽃잎 길이·너비 4개 특성 인풋값에 특성4개 입력
    layers.Dense(16, activation="relu"), #히든레이어 1
    layers.Dense(16, activation="relu"), #히든레이어 2
    layers.Dense(3, activation="softmax")  # 출력층 클래스 3개
])#Dense는 뉴런
#차원 수에따라 히든레이어 지정후 1개는 출력층으로 지정

2025-11-28 15:47:27.635455: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Pro
2025-11-28 15:47:27.635515: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-11-28 15:47:27.635531: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.92 GB
2025-11-28 15:47:27.635582: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-11-28 15:47:27.635599: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [14]:

# 5. 모델 컴파일
model.compile(
    optimizer="adam",                      # 옵티마이저(Optimizer) Adam 경사하강법 방법
    loss="sparse_categorical_crossentropy",# 손실 함수(Loss) – 정수 인코딩 스칼라값 손실함수.다중클래스 / 
    metrics=["accuracy"]                   # 정확도(Accuracy) 측정
)

# 6. EarlyStopping 콜백(과적합 방지)
early_stop = keras.callbacks.EarlyStopping( #keras모델이 검증로스 기준으로 epoch마다 재 계산
    monitor="val_loss", #과적합 감지를 검증로스로 보겠다.
    patience=10, #개선되지않는 에폭 횟수 20번나오면 멈춰라. 
    restore_best_weights=True #그중 성능이 가장좋았던모델 선택
)





In [15]:
# 7. 모델 학습
history = model.fit(
    X_train_scaled, y_train,
    validation_split=0.2,      # 학습 데이터의 20%를 검증(Validation)에 사용
    epochs=100, #200번 반복
    batch_size=6, # 전체 데이터를 /16으로 나눠서 
    callbacks=[early_stop],
    verbose=1
)

Epoch 1/100


2025-11-28 15:55:38.261754: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


16/16 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - accuracy: 0.6042 - loss: 0.7706 - val_accuracy: 0.5833 - val_loss: 0.7306
Epoch 2/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.7396 - loss: 0.5991 - val_accuracy: 0.5833 - val_loss: 0.6182
Epoch 3/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.8021 - loss: 0.5090 - val_accuracy: 0.7083 - val_loss: 0.5518
Epoch 4/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.8333 - loss: 0.4521 - val_accuracy: 0.7500 - val_loss: 0.5068
Epoch 5/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.8542 - loss: 0.4164 - val_accuracy: 0.7917 - val_loss: 0.4704
Epoch 6/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.8646 - loss: 0.3886 - val_accuracy: 0.7917 - val_loss: 0.4435
Epoch 7/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.8646 - loss: 0.3683 - val_accuracy: 0.8333 - val_loss: 0.4180
Epoch 8/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.8750 - loss: 0.3521 - val_accuracy: 0.8750 - val_l

In [16]:
# 8. 테스트 데이터로 평가
test_loss, test_acc = model.evaluate(X_test_scaled, y_test, verbose=0)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.4f}")


Test Loss: 0.0711, Test Accuracy: 0.9667


In [17]:

# 9. 예측(Prediction) 예시
sample = X_test_scaled[:5]
pred_prob = model.predict(sample)          # 각 클래스 확률
pred_class = np.argmax(pred_prob, axis=1)  # 확률이 가장 큰 클래스 선택

print("실제 레이블:", y_test.to_numpy()[:5])
print("예측 레이블:", pred_class)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 183ms/step
실제 레이블: [0 2 1 1 0]
예측 레이블: [0 2 1 1 0]
